In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [ ]:
import json
with open('natural_question.json') as f: 
    data = json.load(f)
len(data)

5249

In [ ]:
question_output = {}
count = 0
for i in range(len(data)):
     if data[str(i+1)][0]['question_template_id'] == '2q7':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if ent[1] == 'new cases' or ent[1] == 'daily cases':
            continue
        print(count)
        print(q)
        print(ent)
        if ent[2] == 'African-American': 
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_black'] = df['Cases_Black'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_black']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_black'] = df['Deaths_Black'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_black']]
                df = df.astype(str)
        elif ent[2] == 'White' or ent[2] == 'Caucasian':
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_white'] = df['Cases_White'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_white']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_white'] = df['Deaths_White'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_white']]
                df = df.astype(str)
        elif ent[2] == 'Asian':
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_asian'] = df['Cases_Asian'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_asian']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_asian'] = df['Deaths_Asian'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_asian']]
                df = df.astype(str)
        elif ent[2] == 'Hispanic' or ent[2] == 'Latino': 
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_latin'] = df['Cases_Latinx'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_latin']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_latin'] = df['Deaths_Latinx'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_latin']]
                df = df.astype(str)
        elif ent[2] == 'American Indian' or ent[2] == 'Alaska Native' or ent[2] == 'American Indian or Alaska Native': 
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_aian'] = df['Cases_AIAN'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_aian']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_aian'] = df['Deaths_AIAN'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_aian']]
                df = df.astype(str)
        elif ent[2] == 'Native Hawaiian' or ent[2] == 'Pacific Islander' or ent[2] == 'Pacific Islander and Native Hawaiian': 
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_nhpi'] = df['Cases_NHPI'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_nhpi']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_nhpi'] = df['Deaths_NHPI'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_nhpi']]
                df = df.astype(str)
        else:
            if ent[1] == 'cases' or ent[1] == 'confirmed cases': 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_cases_multi'] = df['Cases_Multiracial'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_multi']]
                df = df.astype(str)
            else: 
                df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
                df = df.query("Date == '20210307'")
                df['percentage_deaths_multi'] = df['Deaths_Multiracial'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_multi']]
                df = df.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df.columns)
        list_of_list.extend(df.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        res = ""
        count1 = 0
        for i in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[i[0]+1][i[1]]
            else: 
                res += ', ' + list_of_list[i[0]+1][i[1]]
            question_output[q] = res
        count += 1
question_output

0
Which state has the highest percentage of African-American confirmed cases?
['highest', 'confirmed cases', 'African-American']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_black
AK,2.526461268792557
AL,16.563996166612313
AR,15.652457683995346
AS,nan
AZ,3.118745870913566
CA,3.178134194552227
CO,2.8999775106594883
CT,6.887113167209898
DC,48.68297158308989
DE,22.373633338615118



> Which state has the highest percentage of African-American confirmed cases?
DC
1
Which state has the highest percentage of Hispanic confirmed cases?
['highest', 'confirmed cases', 'Hispanic']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_latin
AK,nan
AL,nan
AR,nan
AS,nan
AZ,29.58894263927575
CA,43.10006243227697
CO,27.359889113782295
CT,14.552623278309326
DC,nan
DE,16.447472666772303



> Which state has the highest percentage of Hispanic confirmed cases?
GA, MS
2
Which state has the highest percentage of Latino confirmed cases?
['highest', 'confirmed cases', 'Latino']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...

State,percentage_cases_multi
AK,7.50522483651318
AL,nan
AR,0.5553879403235042
AS,nan
AZ,nan
CA,1.2230557315172186
CO,1.509998577204988
CT,6.183016156730803
DC,nan
DE,nan



> Which state has the least percentage of multiracial cases?
MN
179
Which state has the least percentage of mixed cases?
['least', 'cases', 'mixed']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_multi
AK,7.50522483651318
AL,nan
AR,0.5553879403235042
AS,nan
AZ,nan
CA,1.2230557315172186
CO,1.509998577204988
CT,6.183016156730803
DC,nan
DE,nan



> Which state has the least percentage of mixed cases?
MN


{'Which state has the highest percentage of African-American confirmed cases?': 'DC',
 'Which state has the highest percentage of Hispanic confirmed cases?': 'GA, MS',
 'Which state has the highest percentage of Latino confirmed cases?': 'GA, MS',
 'Which state has the highest percentage of Black confirmed cases?': 'AK',
 'Which state has the highest percentage of Asian confirmed cases?': 'GU',
 'Which state has the highest percentage of American Indian confirmed cases?': 'AZ',
 'Which state has the highest percentage of White confirmed cases?': 'ME',
 'Which state has the highest percentage of Caucasian confirmed cases?': 'ME',
 'Which state has the highest percentage of Native Hawaiian confirmed cases?': 'AK',
 'Which state has the highest percentage of Alaska Native confirmed cases?': 'AK, AZ',
 'Which state has the highest percentage of American Indian or Alaska Native confirmed cases?': 'AK, AZ',
 'Which state has the highest percentage of Pacific Islander confirmed cases?': 'AK',

In [ ]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
df = df.query("Date == '20210307'")
df

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_Latinx,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,...,Tests_Latinx,Tests_Asian,Tests_AIAN,Tests_NHPI,Tests_Multiracial,Tests_Other,Tests_Unknown,Tests_Ethnicity_Hispanic,Tests_Ethnicity_NonHispanic,Tests_Ethnicity_Unknown
0,20210307,AK,59332.0,18300.0,1499.0,NaN,2447.0,12238.0,1508.0,4453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210307,AL,499819.0,160347.0,82790.0,NaN,2273.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210307,AR,324818.0,207596.0,50842.0,NaN,2913.0,1070.0,3358.0,1804.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210307,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210307,AZ,826454.0,308453.0,25775.0,244539.0,11921.0,40707.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20210307,CA,3501394.0,546630.0,111279.0,1509103.0,186562.0,9025.0,15281.0,42824.0,...,9444459.0,3980518.0,98894.0,222513.0,74171.0,6354689.0,18567612.0,9444459.0,21633943.0,18567612.0
6,20210307,CO,435762.0,181669.0,12637.0,119224.0,6406.0,2527.0,1264.0,6580.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20210307,CT,285330.0,85469.0,19651.0,41523.0,3019.0,393.0,NaN,17642.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20210307,DC,41419.0,10708.0,20164.0,NaN,914.0,86.0,82.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20210307,DE,88354.0,42730.0,19768.0,14532.0,1842.0,NaN,NaN,NaN,...,55204.0,16091.0,NaN,NaN,NaN,58282.0,74345.0,55204.0,503875.0,74345.0


In [ ]:
len(question_output)

175

In [ ]:
with open('2q7data.json', 'w') as outfile:
    json.dump(question_output, outfile)